In [1]:
#imports
import numpy as np
import splat
import pandas as pd
import glob
from astropy.io import ascii
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import popsims
from scipy.interpolate import interp1d
#sns.set_style("darkgrid", {"axes.facecolor": ".9"})
import splat
import splat.model as spm
from popsims.relations import polynomial_relation, teff_to_mag_kirkpatrick
import splat.empirical as spe
import astropy.units as u
from scipy.interpolate import griddata
%matplotlib inline



Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [2]:
def read_model_file(f):
    from astropy.io import ascii
    df=ascii.read(f, fast_reader=False,  names=['wavelength', 'surface_flux'] ).to_pandas()
    
    return df

def interpolate_radius(teff, logg, model='burrows1997'):
    model_filename='/volumes/LaCie/popsimsdata/popsims/evmodels/'+model.lower()+'.csv'
    evolutiomodel=pd.read_csv( model_filename)

    #use the full cloud treatment for saumon models
    if model=='saumon2008':
         evolutiomodel=evolutiomodel[evolutiomodel.cloud=='hybrid']
 
    #make age, teff, mass logarithm scale
    valuest=evolutiomodel.temperature.values
    valueslogg=evolutiomodel.gravity.values
    valuesr= evolutiomodel.radius.values
    #valueslumn=evolutiomodel.luminosity.values

    #valuesm=np.log10(evolutiomodel.mass.values)
    #valuesag=np.log10(evolutiomodel.age.values)
    evolpoints=np.array([valuest, valueslogg ]).T

    return griddata(evolpoints, valuesr , (teff, logg), method='linear')

def compute_mag_from_teff(teff):
    return teff_to_mag_kirkpatrick(teff, nsample=1000)

def compute_mag_from_type(spt):
    return spe.typeToMag(spt, '2MASS J')[0]

def get_model_parameters_ger(f):
    teff=float(f.split('/')[-1].split('_')[1])
    logg= float((f.split('/')[-1].split('_')[-1]).split('.dat')[0])
    mets={'ca1':-1.5, 'ca2': -0.5}
    z= mets[f.split('/')[-1].split('_')[0]]
    return {'filename': f, 'teff': teff, 'logg': logg, 'z':z}

def make_spectrum_from_model_ger(f,  teff, logg):
    #FUNIT0=u.erg/(u.s*u.cm**2*u.A)
    FUNIT1= u.erg/(u.micron*u.cm**2*u.s)
    
    df=read_model_file(f)
    #print (df)
    flux=df.surface_flux.values*FUNIT1
    sp= splat.Spectrum(wave=df.wavelength.values*u.Angstrom,\
                       flux=flux, \
                       noise=1e-5*flux,\
                       name=f.split('/')[-1],
                        flux_unit=FUNIT1)
    sp.toMicron()
    #scale spectrum by radius
    sp.trim([0.01, 3.0])
    sp.normalize()
    radius=  interpolate_radius(teff, logg, model='baraffe2003')
    scalef= (radius*u.Rsun.to(u.pc)/10)**2
    sp.scale(scalef)
    return sp

In [3]:
folder= '/volumes/LaCie/popsimsdata/Gerasimov2022/spectra/*'
files=glob.glob(folder)
df= read_model_file(files[0])
#get_model_parameters_ger(files[0])
f=files[0]

In [4]:
get_model_parameters_ger(f)
params= pd.DataFrame([get_model_parameters_ger(f) for f in files])

In [5]:
params

,filename,teff,logg,z
0,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,1000.0,4.00,-1.5
1,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,1000.0,4.25,-1.5
2,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,1000.0,4.50,-1.5
3,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,1000.0,5.00,-1.5
4,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,1000.0,5.25,-1.5
...,...,...,...,...
812,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,900.0,5.00,-0.5
813,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,900.0,5.25,-0.5
814,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,900.0,5.50,-0.5
815,/volumes/LaCie/popsimsdata/Gerasimov2022/spect...,900.0,5.75,-0.5


In [6]:
spectra= [make_spectrum_from_model_ger(f, teff, logg) for f, teff, logg in zip(params.filename.values,\
                                                                              params.teff.values, \
                                                                           params.logg.values) ]#

In [11]:
data={'spectra': spectra,\
      'teff':  params.teff.values,\
      'logg': params.logg.values, 
        'z': params.z.values}

np.save('/Volumes/LaCie/popsimsdata/subdwarfs_models_gerasimov.npy', [data], allow_pickle=True) 